# Task 5: Model Interpretability

In [ ]:
%pip install transformers shap lime torch --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import shap
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from lime.lime_text import LimeTextExplainer
import matplotlib.pyplot as plt

In [9]:
# Load your fine-tuned model and tokenizer
model_path = "amharic-ner-model"  # Replace with your saved path
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForTokenClassification.from_pretrained(model_path)
model.eval()

# Create NER pipeline for predictions
ner_pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Sample input
text = "የመለኪያ ሳሙና በ150 ብር በአዲስ አበባ ይሸጣል።"

# ------------ SHAP Visualization ------------
print("SHAP Interpretation")
def shap_predict(texts):
    probs = []
    for t in texts:
        inputs = tokenizer(t, return_tensors="pt", truncation=True)
        with torch.no_grad():
            outputs = model(**inputs).logits
        probs.append(torch.nn.functional.softmax(outputs, dim=-1)[0].mean(dim=0).cpu().numpy())
    return np.array(probs)

try:
    shap_explainer = shap.Explainer(shap_predict, tokenizer)
    shap_values = shap_explainer([text])
    shap.plots.text(shap_values[0])
except Exception as e:
    print(f"[SHAP ERROR] {e}")

# ------------ LIME Visualization ------------
print(" LIME Interpretation")

def lime_predict(texts):
    results = []
    for t in texts:
        inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            logits = model(**inputs).logits
        probs = torch.nn.functional.softmax(logits, dim=-1)[0].mean(dim=0).numpy()
        results.append(probs)
    return np.array(results)

try:
    class_names = list(model.config.id2label.values())
    lime_explainer = LimeTextExplainer(class_names=class_names)
    lime_exp = lime_explainer.explain_instance(text, lime_predict, num_features=8)
    lime_exp.show_in_notebook(text=True)
except Exception as e:
    print(f"[LIME ERROR] {e}")

# Show model output for context
print(" Model NER Prediction:")
for ent in ner_pipe(text):
    print(ent)

NameError: name 'model' is not defined